In [228]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
load_dotenv()
import os 
os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")
model_name=os.getenv("MODEL")

Model_vision=ChatGoogleGenerativeAI(model=model_name)

In [ ]:
from typing import Any, Dict, List, Union, Optional
from pydantic import BaseModel, Field
import re

def json_to_pydantic_model(json_data: Union[List, Dict], class_name: str = "Data") -> str:
    """
    Convert a list of field definitions to a Pydantic BaseModel class definition.
    
    Args:
        json_data: List of field definitions [name, type, description] or dict
        class_name: Name for the Pydantic class
        
    Returns:
        String containing the Pydantic class definition
    """
    # Handle list of field definitions
    if isinstance(json_data, list):
        fields = []
        for field_def in json_data:
            if len(field_def) >= 3:
                field_name = field_def[0]
                field_type = field_def[1]
                description = field_def[2]
                fields.append((field_name, field_type, description))
            else:
                return "Error: Each field definition should have name, type, and description."
    else:
        return "Error: Expected a list of field definitions."
    
    # Generate class definition
    class_definition = [
        "from pydantic import BaseModel, Field",
        "from typing import List, Optional, Dict, Any, Union\n",
        f"class {class_name}(BaseModel):"
    ]
    
    for field_name, field_type, description in fields:
        # Sanitize field name to be a valid Python identifier
        field_name = re.sub(r'\W|^(?=\d)', '_', field_name)
        
        # Add field with type annotation and Field description
        class_definition.append(f"    {field_name}: {field_type} = Field(default=None, description=\"{description}\")")
    
    return "\n".join(class_definition)



In [280]:
# name
#['str','int','float','bool','List[str]']
# Disction


json_example=[
    ["name","str","name of the person"],
    ["price","int","price for a service "],
    ["gender","bool","return true if he is male"],
    ['list of sujest',"List[str]","list of subjest name"]
]
# Generate the Pydantic class code
pydantic_class_code = json_to_pydantic_model(json_example)
# Execute the generated code to define the class
exec(pydantic_class_code)
print(pydantic_class_code)
strut_llm=Model_vision.with_structured_output(Data)


from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any, Union

class Data(BaseModel):
    name: str = Field( description="name of the person")
    price: int = Field( description="price for a service ")
    gender: bool = Field( description="return true if he is male")
    list_of_sujest: List[str] = Field( description="list of subjest name")


In [ ]:
strut_llm.invoke()

In [ ]:
import base64

# Read local file using regular file operations
image_path = r"C:\Users\AkshayKumarBM\Documents\multimodel_extration\data\raw\ews-letter-1.jpg"

def GetImageDes(image_path, Model):
    
    with open(image_path, 'rb') as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")

    message = HumanMessage(
        content=[
            {"type": "text", "text": "Act as Image Analyst. Analyze and extract insights from images. An expert in visual content interpretation and text Extraction with years of experience in image analysis "},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
            },
            
        ],
    )
    response = Model.invoke([message])
    # print(response.content)
    return response.content


In [318]:
image_des=GetImageDes(image_path,Model_vision)
print(image_des)


Here are the insights from the image:

**Overall**:

*   The image is an official circular from the Commissionerate of Revenue Administration and Disaster Management.
*   The subject is related to the income and asset certificate for the Economic Weaker Section (EWS).
*   The circular instructs to withdraw previous instructions.

**Details:**

*   **Sender**: Dr. J. Radhakrishnan, I.A.S, Principal Secretary/Commissioner of Revenue Administration, Chennai.
*   **Recipients**: All District Collectors and The Commissioner, Tamil Nadu e-Governance Agency.
*   **Date**: June 4, 2020.
*   **Letter Number**: RA 5 (3) / 40521/2017.
*   **Subject**: Income and asset certificate for the Economic Weaker Section - revised instructions.
*   **References**:
    *   This office Circular RA5(3)/40521/2017, dated 04.05.2019.
    *   Government letter No: 41383/RA 3(2)/2019-1, dated 22.11.2019.
    *   This office letter number RA 5(3)/40521/2017, dated 17.12.2019.
    *   Government letter No:41383/RA3

In [319]:
image_path=r"C:\Users\AkshayKumarBM\Documents\multimodel_extration\data\raw\adhar-india-pixlab.jpg"
image_des=GetImageDes(image_path,Model_vision)
resutt=strut_llm.invoke(image_des)

In [320]:
resutt

Data(name='Furkan', price=200, gender=True, list_of_sujest=['Math', 'Science'])